In [ ]:
# storing and retrieving data
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy.integrate import solve_ivp
from scipy import integrate
from matplotlib.pyplot import figure
from itertools import repeat
import seaborn as sn
import ordpy
import pandas as pd
import array

alpha = 0
alpha_array_inner_layer = []
alpha_array_outer_layer = []
beta_array = []
PE_x1_list = []
PE_y1_list = []
PE_x2_list = []
PE_y2_list = []
beta = 0
alpha_max = 2.0
beta_max = 0.4
beta_repeat_times = int((beta_max-beta)/0.005)

while beta <= beta_max:
    while alpha <= alpha_max:
        alpha_array_inner_layer.append(alpha)
        alpha += 0.025
    beta_array.extend(repeat(beta,beta_repeat_times))
    alpha_array_outer_layer.extend(alpha_array_inner_layer)
    beta += 0.005
    
print(len(beta_array), len(alpha_array_outer_layer))
    
xi_0 = 0.175
T_1_ = 5
xi_P = 0.175
A_P = 0.01
omega_P = 0.7
A_0 = 0.5
omega_T_ = 0.7
c = 0.1
b_0 = 0.6
a = 0.7
T_2_ = 5
g = 0.1

def JJ (t, y, xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g):
    x1, y1, x2, y2, z = y
    dx1 = x1*(1-xi_0*math.e**(-1/T_1_)-xi_P)-y1-1/3*x1**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_1_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy1 = c*(-b_0*math.e**(1/T_1_)*y1+a+x1)
    dx2 = x2*(1-xi_0*math.e**(-1/T_2_)-xi_P)-y2-1/3*x2**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_2_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy2 = c*(-b_0*math.e**(1/T_2_)*y2+a+x2)
    dz = g*(x1-x2)
    dydt = [dx1, dy1, dx2, dy2, dz]
    return dydt

tStart = 0
tEnd = 8000

X0 = [0.2, 0.1, 0.02, 0.01, 0.001]

k = 0
while k < len(beta_array) and k < len(alpha_array_outer_layer):
    alpha = alpha_array_outer_layer[k]
    beta = beta_array[k]
    
    solution = integrate.solve_ivp(JJ, [tStart, tEnd], X0, method='RK45', t_eval=np.linspace(tStart,tEnd,100000), args=(xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g))
    
    t_sol = solution.t
    x1_sol = solution.y[0]
    y1_sol = solution.y[1]
    x2_sol = solution.y[2]
    y2_sol = solution.y[3]
    z_sol = solution.y[4]
    
    # permutation entropy of x1
    PE_x1 = ordpy.permutation_entropy(x1_sol, dx=4, taux=1, base=2, normalized=True)
    PE_x1_list.append(PE_x1)
    
    # permutation entropy of y1
    PE_y1 = ordpy.permutation_entropy(y1_sol, dx=4, taux=1, base=2, normalized=True)
    PE_y1_list.append(PE_y1)
    
    # permutation entropy of x2
    PE_x2 = ordpy.permutation_entropy(x2_sol, dx=4, taux=1, base=2, normalized=True)
    PE_x2_list.append(PE_x2)
    
    # permutation entropy of y2
    PE_y2 = ordpy.permutation_entropy(y2_sol, dx=4, taux=1, base=2, normalized=True)
    PE_y2_list.append(PE_y2)
    
    k += 1

PE_x1_matrix = np.array_split(PE_x1_list, beta_repeat_times)
PE_y1_matrix = np.array_split(PE_y1_list, beta_repeat_times)
PE_x2_matrix = np.array_split(PE_x2_list, beta_repeat_times)
PE_y2_matrix = np.array_split(PE_y2_list, beta_repeat_times)

from numpy import savetxt
savetxt('PE_x1.csv', PE_x1_matrix, delimiter=',')
savetxt('PE_y1.csv', PE_y1_matrix, delimiter=',')
savetxt('PE_x2.csv', PE_x2_matrix, delimiter=',')
savetxt('PE_y2.csv', PE_y2_matrix, delimiter=',')

from numpy import loadtxt
PE_x1_matrix_opened = loadtxt('PE_x1.csv', delimiter=',')
PE_y1_matrix_opened = loadtxt('PE_y1.csv', delimiter=',')
PE_x2_matrix_opened = loadtxt('PE_x2.csv', delimiter=',')
PE_y2_matrix_opened = loadtxt('PE_y2.csv', delimiter=',')

In [ ]:
# heatmap for x1
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

con_PE_x1 = np.vstack(PE_x1_matrix_opened)
sn.set(rc = {'figure.figsize':(150, 150)})
heatmap_PE_x1 = sn.heatmap(data=con_PE_x1, annot=True, annot_kws={"size":20})
# alpha tick labels
alpha_ticklabels = 0
list_alpha_ticklabels = []
while alpha_ticklabels <= 1.975:
    list_alpha_ticklabels.append('{:,}'.format(round(alpha_ticklabels, 5)))
    alpha_ticklabels += 0.025
# beta tick labels
beta_ticklabels = 0
list_beta_ticklabels = []
while beta_ticklabels <= 0.4:
    list_beta_ticklabels.append('{:,}'.format(round(beta_ticklabels, 5)))
    beta_ticklabels += 0.005
# heatmap plotting
heatmap_PE_x1.set_xticklabels(list_alpha_ticklabels, fontsize=25)
heatmap_PE_x1.set_yticklabels(list_beta_ticklabels, fontsize=25)
heatmap_PE_x1.invert_yaxis()
heatmap_PE_x1.set_title('Permutation Entropy of x1', fontsize=60)
heatmap_PE_x1.set_xlabel('alpha', fontsize=60)
heatmap_PE_x1.set_ylabel('beta', fontsize=60)
cbar = heatmap_PE_x1.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)
plt.show()
print(heatmap_PE_x1)

In [ ]:
# heatmap for y1
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

con_PE_y1 = np.vstack(PE_y1_matrix_opened)
sn.set(rc = {'figure.figsize':(150, 150)})
heatmap_PE_y1 = sn.heatmap(data=con_PE_y1, annot=True, annot_kws={"size":20})
# alpha tick labels
alpha_ticklabels = 0
list_alpha_ticklabels = []
while alpha_ticklabels <= 1.975:
    list_alpha_ticklabels.append('{:,}'.format(round(alpha_ticklabels, 5)))
    alpha_ticklabels += 0.025
# beta tick labels
beta_ticklabels = 0
list_beta_ticklabels = []
while beta_ticklabels <= 0.4:
    list_beta_ticklabels.append('{:,}'.format(round(beta_ticklabels, 5)))
    beta_ticklabels += 0.005
# heatmap plotting
heatmap_PE_y1.set_xticklabels(list_alpha_ticklabels, fontsize=25)
heatmap_PE_y1.set_yticklabels(list_beta_ticklabels, fontsize=25)
heatmap_PE_y1.invert_yaxis()
heatmap_PE_y1.set_title('Permutation Entropy of y1', fontsize=60)
heatmap_PE_y1.set_xlabel('alpha', fontsize=60)
heatmap_PE_y1.set_ylabel('beta', fontsize=60)
cbar = heatmap_PE_y1.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)
plt.show()
print(heatmap_PE_y1)

In [ ]:
# heatmap for x2
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

con_PE_x2 = np.vstack(PE_x2_matrix_opened)
sn.set(rc = {'figure.figsize':(150, 150)})
heatmap_PE_x2 = sn.heatmap(data=con_PE_x2, annot=True, annot_kws={"size":20})
# alpha tick labels
alpha_ticklabels = 0
list_alpha_ticklabels = []
while alpha_ticklabels <= 1.975:
    list_alpha_ticklabels.append('{:,}'.format(round(alpha_ticklabels, 5)))
    alpha_ticklabels += 0.025
# beta tick labels
beta_ticklabels = 0
list_beta_ticklabels = []
while beta_ticklabels <= 0.4:
    list_beta_ticklabels.append('{:,}'.format(round(beta_ticklabels, 5)))
    beta_ticklabels += 0.005
# heatmap plotting
heatmap_PE_x2.set_xticklabels(list_alpha_ticklabels, fontsize=25)
heatmap_PE_x2.set_yticklabels(list_beta_ticklabels, fontsize=25)
heatmap_PE_x2.invert_yaxis()
heatmap_PE_x2.set_title('Permutation Entropy of x2', fontsize=60)
heatmap_PE_x2.set_xlabel('alpha', fontsize=60)
heatmap_PE_x2.set_ylabel('beta', fontsize=60)
cbar = heatmap_PE_x2.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)
plt.show()
print(heatmap_PE_x2)

In [ ]:
# heatmap for y2
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

con_PE_y2 = np.vstack(PE_y2_matrix_opened)
sn.set(rc = {'figure.figsize':(150, 150)})
heatmap_PE_y2 = sn.heatmap(data=con_PE_y2, annot=True, annot_kws={"size":20})
# alpha tick labels
alpha_ticklabels = 0
list_alpha_ticklabels = []
while alpha_ticklabels <= 1.975:
    list_alpha_ticklabels.append('{:,}'.format(round(alpha_ticklabels, 5)))
    alpha_ticklabels += 0.025
# beta tick labels
beta_ticklabels = 0
list_beta_ticklabels = []
while beta_ticklabels <= 0.4:
    list_beta_ticklabels.append('{:,}'.format(round(beta_ticklabels, 5)))
    beta_ticklabels += 0.005
# heatmap plotting
heatmap_PE_y2.set_xticklabels(list_alpha_ticklabels, fontsize=25)
heatmap_PE_y2.set_yticklabels(list_beta_ticklabels, fontsize=25)
heatmap_PE_y2.invert_yaxis()
heatmap_PE_y2.set_title('Permutation Entropy of y2', fontsize=60)
heatmap_PE_y2.set_xlabel('alpha', fontsize=60)
heatmap_PE_y2.set_ylabel('beta', fontsize=60)
cbar = heatmap_PE_y2.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)
plt.show()
print(heatmap_PE_y2)